<a href="https://colab.research.google.com/github/snehasrini99/Network_attack_classification/blob/main/ML_modeling_UNSW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ML Modeling_UNSW
running different ML algorithms with UNSW dataset to sort which ML algorithm works best with Network dataset

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import statsmodels.api as sm
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree  import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
from sklearn import svm
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import RocCurveDisplay #changed From sklearn 1.2
from sklearn.model_selection import GridSearchCV
from mlxtend.plotting import plot_confusion_matrix

# Turn off the warnings.
warnings.filterwarnings(action='ignore')
%matplotlib inline

In [2]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
%cd drive/MyDrive/24Spring/NSP/Project

## Cleaned UNSW Data
need to run cleaned unsw dataset
(change path/name of dataset)
(change if column names are different - data encoding & splitting)

In [ ]:
Trained_Data_UNSW = pd.read_csv("/content/drive/MyDrive/24Spring/NSP/Project/Data_final/Cleaned_Trained_Data_UNSW.csv" , sep = "," , encoding = 'utf-8')
Tested_Data_UNSW = pd.read_csv("/content/drive/MyDrive/24Spring/NSP/Project/Data_final/Cleaned_Tested_Data_UNSW.csv" , sep = "," , encoding = 'utf-8')

In [ ]:
Trained_Data_UNSW.info()

In [ ]:
Tested_Data_UNSW.info()

### Data Encoding

In [ ]:
Trained_Data_UNSW = pd.get_dummies(Trained_Data_UNSW,columns=['proto','service','state'],prefix="",prefix_sep="")
Tested_Data_UNSW = pd.get_dummies(Tested_Data_UNSW,columns=['proto','service','state'],prefix="",prefix_sep="")

In [ ]:
LE = LabelEncoder()
attack_LE= LabelEncoder()
Trained_Data_UNSW["attack_cat"] = attack_LE.fit_transform(Trained_Data_UNSW["attack_cat"])
Tested_Data_UNSW["attack_cat"] = attack_LE.fit_transform(Tested_Data_UNSW["attack_cat"])

### Data Splitting

In [ ]:
X_train_UNSW = Trained_Data_UNSW.drop(['attack_cat', 'label', 'ct_dst_ltm'], axis=1)
X_test_UNSW = Tested_Data_UNSW.drop(['attack_cat', 'label', 'ct_dst_ltm'], axis=1)


Y_train_UNSW = Trained_Data_UNSW['label']
Y_test_UNSW = Tested_Data_UNSW['label']

X_train_train_UNSW,X_test_train_UNSW ,Y_train_train_UNSW,Y_test_train_UNSW = train_test_split(X_train_UNSW, Y_train_UNSW, test_size= 0.25 , random_state=42)
X_train_test_UNSW,X_test_test_UNSW,Y_train_test_UNSW,Y_test_test_UNSW = train_test_split(X_test_UNSW, Y_test_UNSW, test_size= 0.25 , random_state=42)

### Data Scaling

In [ ]:
Ro_scaler = RobustScaler()
X_train_train_UNSW = Ro_scaler.fit_transform(X_train_train_UNSW)
X_test_train_UNSW = Ro_scaler.transform(X_test_train_UNSW)
X_train_test_UNSW = Ro_scaler.fit_transform(X_train_test_UNSW)
X_test_test_UNSW = Ro_scaler.transform(X_test_test_UNSW)

In [ ]:
X_train_train_UNSW.shape, Y_train_train_UNSW.shape
X_test_train_UNSW.shape, Y_test_train_UNSW.shape
X_train_test_UNSW.shape, Y_train_test_UNSW.shape
X_test_test_UNSW.shape, Y_test_test_UNSW.shape

### VIF

In [ ]:
A = sm.add_constant(X_train_UNSW.astype(float))
Est1 = sm.GLM(Y_train_UNSW, A)
Est2 = Est1.fit()
Est2.summary()

### Data Modeling

In [ ]:
def Evaluate(Model_Name, Model_Abb, X_test, Y_test):

    Pred_Value= Model_Abb.predict(X_test)
    Accuracy = metrics.accuracy_score(Y_test,Pred_Value)
    Sensitivity = metrics.recall_score(Y_test,Pred_Value)
    Precision = metrics.precision_score(Y_test,Pred_Value)
    F1_score = metrics.f1_score(Y_test,Pred_Value)
    Recall = metrics.recall_score(Y_test,Pred_Value)

    print('--------------------------------------------------\n')
    print('The {} Model Accuracy   = {}\n'.format(Model_Name, np.round(Accuracy,3)))
    print('The {} Model Sensitvity = {}\n'.format(Model_Name, np.round(Sensitivity,3)))
    print('The {} Model Precision  = {}\n'.format(Model_Name, np.round(Precision,3)))
    print('The {} Model F1 Score   = {}\n'.format(Model_Name, np.round(F1_score,3)))
    print('The {} Model Recall     = {}\n'.format(Model_Name, np.round(Recall,3)))
    print('--------------------------------------------------\n')

    Confusion_Matrix = metrics.confusion_matrix(Y_test, Pred_Value)
    plot_confusion_matrix(Confusion_Matrix,class_names=['Normal', 'Attack'],figsize=(5.55,5), colorbar= "blue")

    # changed into RocCurveDisplay https://github.com/mrdbourke/zero-to-mastery-ml/issues/45
    # RocCurveDisplay.from_estimator(estimator=gs_log_reg, X=X_test, y=y_test);
    RocCurveDisplay.from_estimator(estimator=Model_Abb, X=X_test, y=Y_test)

    return X_test, Y_test, Pred_Value

In [ ]:
def GridSearch(Model_Abb, Parameters, X_train, Y_train):
    Grid = GridSearchCV(estimator=Model_Abb, param_grid= Parameters, cv = 3, n_jobs=-1)
    Grid_Result = Grid.fit(X_train, Y_train)
    Model_Name = Grid_Result.best_estimator_

    return (Model_Name)

#### Logistic Regression

In [ ]:
LR_UNSW = LogisticRegression()
LR_UNSW.fit(X_train_train_UNSW , Y_train_train_UNSW)

In [ ]:
X_test_UNSW_LR, Y_test_UNSW_LR, Pred_Value_UNSW_LR = Evaluate('Logistic Regression', LR_UNSW, X_test_train_UNSW, Y_test_train_UNSW)

#### Decision Tree

In [ ]:
DT_UNSW = DecisionTreeClassifier(max_features=6, max_depth=4)
DT_UNSW.fit(X_train_train_UNSW, Y_train_train_UNSW)

In [ ]:
X_test_UNSW_DT, Y_test_UNSW_DT, Pred_Value_UNSW_DT = Evaluate('Decision Tree Classifier', DT_UNSW, X_test_train_UNSW, Y_test_train_UNSW)

#### Random Forest

In [ ]:
max_depth= [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
Parameters={ 'max_depth': max_depth}

In [ ]:
RF_UNSW = RandomForestClassifier()
GridSearch(RF_UNSW, Parameters, X_train_train_UNSW, Y_train_train_UNSW)
RF_UNSW.fit(X_train_train_UNSW, Y_train_train_UNSW)

In [ ]:
X_test_UNSW_RF, Y_test_UNSW_RF, Pred_Value_UNSW_RF = Evaluate('Random Forest Classifier', RF_UNSW, X_test_train_UNSW, Y_test_train_UNSW)

#### KNN

In [ ]:
KNN_UNSW = KNeighborsClassifier(n_neighbors=6)
KNN_UNSW.fit(X_train_train_UNSW, Y_train_train_UNSW)

In [ ]:
X_test_UNSW_KNN, Y_test_UNSW_KNN, Pred_Value_UNSW_KNN = Evaluate('KNN', KNN_UNSW, X_test_train_UNSW, Y_test_train_UNSW)

#### SVM

In [ ]:
Linear_SVC_UNSW = svm.LinearSVC(C=1)
Linear_SVC_UNSW.fit(X_train_train_UNSW, Y_train_train_UNSW)

In [ ]:
X_test_UNSW_SVM, Y_test_UNSW_SVM, Pred_Value_UNSW_SVM = Evaluate('SVM Linear SVC Kernel', Linear_SVC_UNSW, X_test_train_UNSW, Y_test_train_UNSW)

# Saving results to CSV
for further features on classifying which type of attack it was, we are saving the results from the Best Performed model using UNSW dataset

In [ ]:
def result(X_test, Y_test, Pred_Value):
# (122column, id as index, attack_state)

    # df_X_test = pd.DataFrame(X_test)
    df_Pred_Value_m = pd.Series(Pred_Value).rename('pred')
    m_Y_test = Y_test.reset_index()
    m_Y_test.columns = ['id', 'attack_state']

    # result = pd.concat([df_X_test, m_Y_test, df_Pred_Value_m], axis=1)
    result = pd.concat([m_Y_test, df_Pred_Value_m], axis=1)
    result = result.set_index('id')

    # count correct rows
    matching_df = (result['attack_state'] == result['pred']).sum()
    print(f"number of correct perdiction is: {matching_df}")
    print(f"total number of rows are: {len(X_test)}")

    # filter attacks
    filtered_df = result[result['attack_state'] == 1]
    print(f"number of rows that are attack is: {len(filtered_df)}")
    filtered_df

    return result, filtered_df

In [ ]:
# Logistic Regression
result_UNSW_lr, filtered_UNSW_lr = result(X_test_UNSW_LR, Y_test_UNSW_LR, Pred_Value_UNSW_LR)
result_UNSW_lr.to_csv(r'/content/drive/MyDrive/24Spring/NSP/Project/outcome/modeling_result_unsw_lr.csv',index = False)
filtered_UNSW_lr.to_csv(r'/content/drive/MyDrive/24Spring/NSP/Project/outcome/modeling_attacks_unsw_lr.csv',index = False)

In [ ]:
# Decision Tree
result_UNSW_dt, filtered_UNSW_dt = result(X_test_UNSW_DT, Y_test_UNSW_DT, Pred_Value_UNSW_DT)
result_UNSW_dt.to_csv(r'/content/drive/MyDrive/24Spring/NSP/Project/outcome/modeling_result_unsw_dt.csv',index = False)
filtered_UNSW_dt.to_csv(r'/content/drive/MyDrive/24Spring/NSP/Project/outcome/modeling_attacks_unsw_dt.csv',index = False)

In [ ]:
# Random Forest
result_UNSW_rf, filtered_UNSW_rf = result(X_test_UNSW_RF, Y_test_UNSW_RF, Pred_Value_UNSW_RF)
result_UNSW_rf.to_csv(r'/content/drive/MyDrive/24Spring/NSP/Project/outcome/modeling_result_unsw_rf.csv',index = False)
filtered_UNSW_rf.to_csv(r'/content/drive/MyDrive/24Spring/NSP/Project/outcome/modeling_attacks_unsw_rf.csv',index = False)

In [ ]:
# KNN
result_UNSW_knn, filtered_UNSW_knn = result(X_test_UNSW_KNN, Y_test_UNSW_KNN, Pred_Value_UNSW_KNN)
result_UNSW_knn.to_csv(r'/content/drive/MyDrive/24Spring/NSP/Project/outcome/modeling_result_unsw_knn.csv',index = False)
filtered_UNSW_knn.to_csv(r'/content/drive/MyDrive/24Spring/NSP/Project/outcome/modeling_attacks_unsw_knn.csv',index = False)

In [ ]:
# SVM
result_UNSW_svm, filtered_UNSW_svm = result(X_test_UNSW_SVM, Y_test_UNSW_SVM, Pred_Value_UNSW_SVM)
result_UNSW_svm.to_csv(r'/content/drive/MyDrive/24Spring/NSP/Project/outcome/modeling_result_unsw_svm.csv',index = False)
filtered_UNSW_svm.to_csv(r'/content/drive/MyDrive/24Spring/NSP/Project/outcome/modeling_attacks_unsw_svm.csv',index = False)